In [ ]:
!pip install streamlit opencv-python torch torchvision pillow pyngrok
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/PCB_Dataset/efficientnet_pcb.pth /content/efficientnet_pcb.pth

In [ ]:
%%writefile app.py

import streamlit as st
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b0
import torch.nn as nn
import tempfile
import os


# ================= CONFIG =================

CLASSES = [
    "Missing_hole",
    "Mouse_bite",
    "Open_circuit",
    "Short",
    "Spur",
    "Spurious_copper"
]

MODEL_PATH = "efficientnet_pcb.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ================= CHECK MODEL =================

if not os.path.exists(MODEL_PATH):
    st.error("Model file not found: efficientnet_pcb.pth")
    st.stop()


# ================= LOAD MODEL =================

@st.cache_resource
def load_model():

    model = efficientnet_b0()

    model.classifier[1] = nn.Linear(
        model.classifier[1].in_features,
        len(CLASSES)
    )

    model.load_state_dict(
        torch.load(MODEL_PATH, map_location=device)
    )

    model.to(device)
    model.eval()

    return model


model = load_model()


# ================= TRANSFORM =================

transform = transforms.Compose([

    transforms.Grayscale(3),
    transforms.Resize((224,224)),
    transforms.ToTensor(),

    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std =[0.229,0.224,0.225]
    )
])


# ================= IMAGE ALIGN =================

def align_images(template, test):

    orb = cv2.ORB_create(3000)

    kp1, des1 = orb.detectAndCompute(template, None)
    kp2, des2 = orb.detectAndCompute(test, None)

    if des1 is None or des2 is None:
        return test

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(des1, des2)

    if len(matches) < 15:
        return test

    matches = sorted(matches, key=lambda x: x.distance)

    good = matches[:150]

    pts1 = np.float32([kp1[m.queryIdx].pt for m in good])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good])

    H, _ = cv2.findHomography(pts2, pts1, cv2.RANSAC)

    if H is None:
        return test

    h, w = template.shape

    aligned = cv2.warpPerspective(test, H, (w, h))

    return aligned


# ================= DEFECT DETECTION =================

def detect_defects(template, test):

    # Resize
    template = cv2.resize(
        template,
        (test.shape[1], test.shape[0])
    )

    # Blur (noise reduction)
    template_blur = cv2.GaussianBlur(template, (5,5), 0)
    test_blur     = cv2.GaussianBlur(test, (5,5), 0)

    # Absolute difference
    diff = cv2.absdiff(test_blur, template_blur)

    # OTSU threshold
    _, mask = cv2.threshold(
        diff,
        0,
        255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    # Morphological cleaning
    kernel = np.ones((3,3), np.uint8)

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(
        mask,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    rois = []
    boxes = []

    for c in contours:

        # Strong filtering (reduce false positives)
        if cv2.contourArea(c) > 200:

            x,y,w,h = cv2.boundingRect(c)

            # Ignore very thin regions
            if w < 10 or h < 10:
                continue

            roi = test[y:y+h, x:x+w]

            rois.append(roi)
            boxes.append((x,y,w,h))

    return rois, boxes, mask


# ================= CLASSIFICATION =================

def classify(model, rois, conf_thresh=0.5):

    preds = []
    scores = []

    for roi in rois:

        pil = Image.fromarray(roi)

        img = transform(pil).unsqueeze(0).to(device)

        with torch.no_grad():

            out = model(img)
            prob = torch.softmax(out, dim=1)

        conf, pred = torch.max(prob, 1)

        conf = conf.item()
        pred = pred.item()

        if conf < conf_thresh:

            preds.append("background")
            scores.append(conf)

        else:

            preds.append(CLASSES[pred])
            scores.append(conf)

    return preds, scores


# ================= MAIN PIPELINE =================

def run_pipeline(template_path, test_path):

    # Read grayscale (as in original)
    template = cv2.imread(template_path, 0)
    test     = cv2.imread(test_path, 0)

    # Align
    test = align_images(template, test)

    # Detect
    rois, boxes, mask = detect_defects(template, test)

    # Debug
    st.write("Detected ROIs:", len(rois))
    st.image(mask, caption="OTSU Difference Mask")

    preds, scores = classify(model, rois)

    vis = cv2.cvtColor(test, cv2.COLOR_GRAY2BGR)

    count = 0

    for (x,y,w,h), p, s in zip(boxes, preds, scores):

        if p == "background":
            continue

        count += 1

        cv2.rectangle(
            vis,
            (x,y),
            (x+w,y+h),
            (0,255,0),
            2
        )

        text = f"{p} ({s:.2f})"

        cv2.putText(
            vis,
            text,
            (x,y-8),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            (0,0,255),
            2
        )

    return vis, count


# ================= STREAMLIT UI =================

st.set_page_config(
    page_title="PCB Defect Detection",
    layout="wide"
)

st.title("🔧 PCB Defect Detection System")

st.markdown("""
### Reference-Based PCB Defect Detection (OTSU Method)
""")


col1, col2 = st.columns(2)

with col1:
    template_file = st.file_uploader(
        "Upload Template Image",
        type=["jpg","png","jpeg"]
    )

with col2:
    test_file = st.file_uploader(
        "Upload Test Image",
        type=["jpg","png","jpeg"]
    )


if template_file and test_file:

    with tempfile.NamedTemporaryFile(delete=False) as t1:
        t1.write(template_file.read())
        template_path = t1.name

    with tempfile.NamedTemporaryFile(delete=False) as t2:
        t2.write(test_file.read())
        test_path = t2.name

    if st.button("Run Detection"):

        with st.spinner("Processing..."):

            result, total = run_pipeline(
                template_path,
                test_path
            )

        st.success("Detection Completed")

        st.info(f"Detected Defects: {total}")

        st.subheader("Result")

        st.image(
            cv2.cvtColor(result, cv2.COLOR_BGR2RGB),
            use_column_width=True
        )

        save_path = "output.png"

        cv2.imwrite(save_path, result)

        with open(save_path,"rb") as f:

            st.download_button(
                "Download Result",
                f,
                file_name="pcb_result.png"
            )

Overwriting app.py


In [ ]:
import subprocess
import time
import re

# Start Streamlit
print("Starting Streamlit...")
streamlit = subprocess.Popen(
    [
        "streamlit", "run", "app.py",
        "--server.port=8501",
        "--server.address=0.0.0.0",
        "--server.enableCORS=false",
        "--server.enableXsrfProtection=false"
    ],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

time.sleep(10)  # wait for server

print("Starting Cloudflare Tunnel...")

# Start Cloudflare
cloudflared = subprocess.Popen(
    [
        "./cloudflared-linux-amd64",
        "tunnel",
        "--url",
        "http://localhost:8501",
        "--no-autoupdate"
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Regex for URL
url_pattern = re.compile(r"https://.*trycloudflare.com")

# Read output until URL appears
for line in cloudflared.stdout:
    print(line.strip())

    match = url_pattern.search(line)
    if match:
        print("\n" + "="*50)
        print("YOUR PUBLIC APP LINK:")
        print(match.group(0))
        print("="*50 + "\n")
        break

Starting Streamlit...
Starting Cloudflare Tunnel...
2026-02-12T11:28:57Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-02-12T11:28:57Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-02-12T11:29:01Z INF +--------------------------------------------------------------------------------------------+
2026-02-12T11:29:01Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-02-12T11:29:01Z INF |  h